In [ ]:
# Read in the GTex Data pick tissue to use for normalization
# https://gtexportal.org/home/datasets
# This file contains the median TPM by tissue. These medians were calculated directly from the file GTEx_Analysis_2016-01-15_v7_RNASeQCv1.1.8_gene_tpm.gct.gz.